## AIF_sprint18-segmentation1
この課題の目的    
・公開されている実装を動かす際のノウハウを得る    
・論文で読んだ手法を大規模なデータに適用する     

【目的としないこと】   
・高い精度を出す 

### コードリーディング
unet/model.py　14行目に、入力画像(サイズ：256,256,1)をU-Netで処理するコードが書かれている    

①Input     

②down sampling(encorder)：入力画像を低解像度に変換するネットワーク     
⑴conv（conv　３×３　ReLU、paddingは行わない）×２、max pooling ２×２    
⑵各conv、poolの１〜４まで行う（4でdropoutが入る）     

③U-Netのボトムconv5：convを２回（pooling行わない、dropout）      

④up sampling(decorder):入力解像度まで拡大するネットワーク      
⑴merge(encorderとdecorderの特徴マップをconcat(結合)することで、従来通りの特徴を抽出しつつ、位置情報を保持）、conv（conv　３×３　ReLU）×２      
⑵各merge、poolの6〜9まで行う（4でdropoutが入る）     

⑤Output:conv10(Sigmoidをする)   

#unet/model.py
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs) 
    #conv1 = Conv2D(64(チャネル), 3（フィルタのサイズ）, activation = 'relu',
    #padding = 'same'(paddingを行わないため、'valid'が正しい), kernel_initializer（フィルタの初期値） = 'he_normal')(inputs) 
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

### データセットをU-Netで学習し推定
動かしてみたが、traingの途中で、時間切れとなった。

In [1]:
from model import *
from data import *

Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'train','images','masks',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

/Users/andoutakaaki/DIC_study/sprint18/unet-master/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/5
Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
1234/2000 [=================>............] - ETA: 4:06:23 - loss: 0.5200 - acc: 0.7691 

### 説明課題

### セグメンテーション
・セグメンテーションとは、ピクセルごとにそのピクセルがどのクラスに属するかを出力するタスク。    
・セマンティックセグメンテーションは、画像内の物体クラスとその輪郭まで切り分けて認識する。他方、インスタンスセグメンテーションは、対象物体そのものを特定する（物体を１つずつ切り分けて認識）。     
・物体検出は、画像から物体のインスタンスやクラスを予測する画像分類だけでなく、物体が存在する領域（物体の位置）まで推定する方法である。   
・セグメンテーション の手法としては、Fully Convolutional Network(FCN)、SegNet、U-Net、PSPNet、RefineNetなどがある。

### スキップコネクション
ディープラーニングにおけるスキップコネクションとは何か。    
・U-Netのようなエンコーダ・デコーダ構造のセグメンテーション手法においては、エンコーダ時の特徴マップをデコーダ時にも利用するテクニックとして、スキップコネクションを導入することにより、エンコーダ側の特徴マップの切り取り・コピーをしてデコーダ側に渡すことによって、連結を行っている。 U-Netは、エンコーダ時にパディングを行わずに畳み込み処理を行うため、エンコーダ時とデコーダで層が異なり、対となる特徴マップが異なることがあるため、 この場合は、デコーダ時に利用するエンコーダ側の特徴マップの中央部分を切り出して、連結している。  
・ResNet[2]などの画像分類手法においては、深いネットワークを学習させる際、誤差逆伝播法で勾配消失を防ぐために、スキップコネクションを導入することで、超多層化を実現している。ここでのスキップコネクションは、畳み込み層などの処理結果を単純に次の層に渡していくのではなく、その処理結果と入力自体を次の層に渡す構造となっている。